# RNN 

In [31]:
# import

import matplotlib.pyplot as plt


import numpy as np
np.random.seed(42)

import random
random.seed(42)

import pickle

import tensorflow as tf
tf.random.set_seed(42)

# Désactiver cuDNN non déterministes
tf.config.experimental.enable_op_determinism()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error, root_mean_squared_error
from sklearn.model_selection import KFold 


In [32]:
taille_fenetre_to_run = 20
taille_stride_to_run = 5
# 20w_5s déterminer dans search window size 

In [33]:
data = pickle.load(open(f"Data/donnees_3D_{taille_fenetre_to_run}w_{taille_stride_to_run}s.pkl", "rb"))

In [34]:
data.keys()

dict_keys(['X_np_3D', 'y_np', 'features_names_3D', 'idx_explicabilite', 'X_3D_explicabilite', 'y_explicabilite'])

In [35]:
X_3D = data["X_np_3D"]
y_np = data["y_np"]

In [36]:
X_binary_explicabilite = data["X_3D_explicabilite"]
y_explicabilite = data["y_explicabilite"]

In [37]:
features_3D = data["features_names_3D"]

In [38]:
X_binary_explicabilite.shape

(73, 20, 25)

In [39]:
# Paramètres pour la validation croisée
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)

# Listes pour stocker les scores
mse_scores = []
mae_scores = []
mape_scores = []
rmse_scores = []

# Création d'un modèle RNN
def create_rnn(input_shape):
    model = Sequential([
        SimpleRNN(64, activation='tanh', input_shape=input_shape, return_sequences=False),
        Dense(1)  # Sortie pour régression
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
    return model

for fold, (train_index, test_index) in enumerate(kf.split(X_3D)):
    print(f"Fold {fold+1}/{n_splits}")
    X_train, X_test = X_3D[train_index], X_3D[test_index]
    y_train, y_test = y_np[train_index], y_np[test_index]
    
    # Création du modèle
    model = create_rnn(X_train.shape[1:]) 
    
    # Entraînement
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
    
    # Prédictions
    y_pred = model.predict(X_test) #.flatten()
    
    # Calcul des scores
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    
    mse_scores.append(mse)
    mae_scores.append(mae)
    mape_scores.append(mape)
    rmse_scores.append(rmse)
    
    print(f"MSE: {mse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"MAPE: {mape:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print()

    # save the model
    filename = f'Models/CNN_{taille_fenetre_to_run}_fold_{fold}.sav'
    pickle.dump(model, open(filename, 'wb'))


Fold 1/5
Epoch 1/10


/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-01-20 16:09:39.632711: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be 

476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6809.2378 - mse: 6809.2378
Epoch 2/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2764.4863 - mse: 2764.4863
Epoch 3/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 987.4781 - mse: 987.4781 
Epoch 4/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 279.4230 - mse: 279.4230
Epoch 5/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 69.4495 - mse: 69.4495
Epoch 6/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 28.1573 - mse: 28.1573
Epoch 7/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 23.3579 - mse: 23.3579
Epoch 8/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.0489 - mse: 23.0489
Epoch 9/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.0352 - mse: 23.0352
Epoch 10/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 23.0349 - mse: 23.0349
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


2025-01-20 16:09:47.313583: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-01-20 16:09:47.313895: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

MSE: 22.47
MAE: 3.66
MAPE: 0.04
RMSE: 4.74

Fold 2/5
Epoch 1/10


...

MSE: 23.25
MAE: 3.67
MAPE: 0.04
RMSE: 4.82

Fold 4/5
Epoch 1/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6845.1431 - mse: 6845.1431
Epoch 2/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2794.0374 - mse: 2794.0374
Epoch 3/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1003.1039 - mse: 1003.1039
Epoch 4/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 284.9758 - mse: 284.9758
Epoch 5/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 70.4829 - mse: 70.4829
Epoch 6/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 28.1508 - mse: 28.1508
Epoch 7/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.3559 - mse: 23.3559
Epoch 8/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.1082 - mse: 23.1082
Epoch 9/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.1088 - mse: 23.1088
Epoch 10/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.1099 - mse: 23.1099
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 964us/step
MSE: 24.73
MAE: 3.73
MAPE: 0.04
RMSE: 4.97



2025-01-20 16:10:10.403156: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-01-20 16:10:10.403481: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

Fold 5/5
Epoch 1/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6783.7729 - mse: 6783.7729
Epoch 2/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2718.8096 - mse: 2718.8096
Epoch 3/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 962.9227 - mse: 962.9227
Epoch 4/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 269.2981 - mse: 269.2981
Epoch 5/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 66.5406 - mse: 66.5406
Epoch 6/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 27.7490 - mse: 27.7490
Epoch 7/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.5506 - mse: 23.5506
Epoch 8/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 23.3522 - mse: 23.3522
Epoch 9/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 23.3541 - mse: 23.3541
Epoch 10/10
476/476 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 23.3546 - mse: 23.3546
 52/119 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step

2025-01-20 16:10:18.032754: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-01-20 16:10:18.033133: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
MSE: 23.26
MAE: 3.68
MAPE: 0.04
RMSE: 4.82



In [40]:
print(f"Mean MSE: {np.mean(mse_scores)}")
print(f"Mean MAE: {np.mean(mae_scores)}")
print(f"Mean MAPE: {np.mean(mape_scores)}")
print(f"Mean RMSE: {np.mean(rmse_scores)}")

Mean MSE: 23.269949627289595
Mean MAE: 3.671365068598346
Mean MAPE: 0.03960351921140839
Mean RMSE: 4.823200212244748


In [41]:
print(f'MSE scores: {mse_scores}')
print(f'MAE scores: {mae_scores}')
print(f'MAPE scores: {mape_scores}')
print(f'RMSE scores: {rmse_scores}')

MSE scores: [np.float64(22.470299600508753), np.float64(22.638749549855792), np.float64(23.25144060340215), np.float64(24.72679724186621), np.float64(23.262461140815077)]
MAE scores: [np.float64(3.6628367257329764), np.float64(3.626449830280708), np.float64(3.6665297275838697), np.float64(3.72555399928586), np.float64(3.6754550601083165)]
MAPE scores: [np.float64(0.03936165727580582), np.float64(0.03902304640439212), np.float64(0.03954336334847667), np.float64(0.04043588939682243), np.float64(0.03965363963154488)]
RMSE scores: [np.float64(4.740284759432575), np.float64(4.758019498683858), np.float64(4.821974761796473), np.float64(4.972604673796843), np.float64(4.82311736751399)]


In [42]:
# get meilleur fold
best_fold = np.argmin(mse_scores)
print(f"Best fold: {best_fold}")

Best fold: 0


In [43]:
# save results

results = {
    "model_name": "RNN",
    "mse": mse_scores,
    "mae": mae_scores,
    "mape": mape_scores,
    "rmse": rmse_scores,
    "best_fold": best_fold,
    "mean_mse": np.mean(mse_scores),
    "mean_mae": np.mean(mae_scores),
    "mean_mape": np.mean(mape_scores),
    "mean_rmse": np.mean(rmse_scores),
}

pickle.dump(results, open(f"Data_to_share/RNN_results.pkl", "wb"))

In [44]:
pickle.load(open(f"Models/CNN_{taille_fenetre_to_run}_fold_{best_fold}.sav", "rb"))

<Sequential name=sequential_10, built=True>

In [45]:
# pas d'explicabilité -> trop mauvais 